# CFM Challenge
2018-07. 

_Note_ : This model gives lower scores than the actual results on the challenge : 
   - Indeed the notebook will give you around ~21.35 by averaging the 5kfolds which will give around ~20.94 on the public leaderboard. This might be due to high regularisation on the model and also because the test set give better predictions than the train set. 

To get the best score  on the leaderboard at the end of the competition (around ~20.88), I've just averaged the results of the top 10 best csvs results with different models I've tried during the competion.
The different models tried were always pretty the sames. What basically changes is the size of the layers, the concatenation/multiplication/addition in the model, the way you deal with nans in the dataset and what kind of engineered features you will take in input + embeddings size (ok that's a lot of changes... but no matter what you do, you will have more or else the same result with this model).



If you want more details about the model and how does it work, you can find it on the attached slides. This presentation was given at CFM and the end of the academic competition and gives some insights of what worked and what did not.

PS : The script isn't really well coded...  basically because I didn't really take the time to do a whole refactoring and it was made of a lot of iterations. So some stuff should not be done like this anymore and doesn't really make sens.. (sorry for that !) 

If you have any question or suggestions (or if you get way better scores than me !), let me know, I would be happy to discuss and learn ;).


#### Let's import some stuff

In [12]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import datetime
import numpy as np
import pandas as pd

from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.layers import multiply, Layer, LeakyReLU, PReLU, Bidirectional, merge, Dense, GlobalAveragePooling1D, SpatialDropout1D,concatenate,add,Embedding,Input,Masking,LSTM,Flatten,TimeDistributed,GlobalMaxPooling1D,GRU,Activation,Dropout,Reshape, BatchNormalization
from sklearn.preprocessing import LabelEncoder, Imputer, StandardScaler, RobustScaler, MinMaxScaler
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau,TensorBoard, LearningRateScheduler
from keras import losses
from keras import initializers, regularizers, constraints
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold,train_test_split
import keras
import hashlib
import math
import h5py


#These libs are not from me, I've just copy past the code to use them... 
#You can find the original repos below and the papers associated in the code.
from CLR.clr_callback import CyclicLR #https://github.com/bckenstler/CLR/blob/master/clr_callback.py | https://arxiv.org/abs/1506.01186
from JANET.janet import JANET #https://github.com/titu1994/Keras-just-another-network-JANET/blob/master/janet.py | https://arxiv.org/abs/1804.04849
import gc
print(keras.__version__)

import warnings
warnings.filterwarnings("ignore", category=Warning)

2.2.4


#### Get the data

In [13]:
home_dir = "../data/"

id_col = "ID"
target_col = "TARGET"


In [14]:
train_df = pd.read_csv("%s/input_training.csv" % home_dir)
id_df = pd.read_csv("%s/output_training_hTMcdrS.csv" % home_dir)
test_df = pd.read_csv("%s/input_test.csv" % home_dir)
train_df  = train_df.merge(id_df, on="ID")
train_df = train_df.set_index('ID')
del id_df; gc.collect()
print(train_df.head())

     eqt_code        date  09:30:00  09:35:00  09:40:00  09:45:00  09:50:00  \
ID                                                                            
0   975514820  2059277756  0.378421  0.062977 -1.699955 -0.253532  0.317230   
1   975514820   661006643 -0.588441 -0.393182  0.918810  0.392393  0.195901   
2   975514820  2252275018 -0.974472  1.789703 -0.105310 -0.456042  0.894337   
3   975514820   901241465 -4.612982  1.746131  1.092367 -0.030626  0.996415   
4   975514820  1213815424 -2.642804 -0.772708 -0.115318  0.145272 -0.136105   

    09:55:00  10:00:00  10:05:00  ...  14:40:00  14:45:00  14:50:00  14:55:00  \
ID                                ...                                           
0  -1.394081 -1.338014 -0.320252  ... -0.193950  0.129398  0.064666  0.323250   
1  -0.195754  0.000000  0.130601  ... -0.260940  0.000000 -0.130601  0.130666   
2  -0.341168 -0.373679  1.103954  ... -0.236124 -0.256959 -0.229774  0.063974   
3  -1.391209  0.169799 -0.827777  ... -0.

In [15]:
train_df = train_df.rename(index = str, columns = {'end_of_day_return' : 'TARGET'})


In [16]:
return_cols = train_df.columns.values[2:73]
other_cols = ["date" , "eqt_code"]
print(return_cols)
#del train_df; gc.collect()
#del test_df; gc.collect()

['09:30:00' '09:35:00' '09:40:00' '09:45:00' '09:50:00' '09:55:00'
 '10:00:00' '10:05:00' '10:10:00' '10:15:00' '10:20:00' '10:25:00'
 '10:30:00' '10:35:00' '10:40:00' '10:45:00' '10:50:00' '10:55:00'
 '11:00:00' '11:05:00' '11:10:00' '11:15:00' '11:20:00' '11:25:00'
 '11:30:00' '11:35:00' '11:40:00' '11:45:00' '11:50:00' '11:55:00'
 '12:00:00' '12:05:00' '12:10:00' '12:15:00' '12:20:00' '12:25:00'
 '12:30:00' '12:35:00' '12:40:00' '12:45:00' '12:50:00' '12:55:00'
 '13:00:00' '13:05:00' '13:10:00' '13:15:00' '13:20:00' '13:25:00'
 '13:30:00' '13:35:00' '13:40:00' '13:45:00' '13:50:00' '13:55:00'
 '14:00:00' '14:05:00' '14:10:00' '14:15:00' '14:20:00' '14:25:00'
 '14:30:00' '14:35:00' '14:40:00' '14:45:00' '14:50:00' '14:55:00'
 '15:00:00' '15:05:00' '15:10:00' '15:15:00' '15:20:00']


#### Some feature engineering

In [17]:
#sum of nans
for df in [train_df,test_df]:
    df["ret_nan"] = df[return_cols].isnull().sum(axis=1)

In [18]:
#replace nans by interpolation
for df in [train_df,test_df]:
    df[return_cols] = df[return_cols].interpolate(axis=1, limit_direction="both", inplace=False)

In [19]:
def get_mean_groupby(all_data, groupby_cols):
    for groupby_obj in groupby_cols:
        groupby_col = groupby_obj["id"]
        print(groupby_col)
        cols = groupby_obj["cols"]
        group_by = all_data.groupby([groupby_col])
        data_arr = []
        data_arr.append({"i": "avg", "d": group_by[cols].mean()})
        data_arr.append({"i": "skew", "d": group_by[cols].skew()})
        data_arr.append({"i": "kurt", "d": group_by[cols].apply(pd.DataFrame.kurt)})
        data_arr.append({"i": "std", "d": group_by[cols].std()})
        data_arr.append({"i": "median", "d": group_by[cols].median()})
        data_arr.append({"i": "nan", "d": all_data.isnull().groupby(all_data[groupby_col])[cols].sum()})

        all_data.set_index([groupby_col], inplace=True)
        for obj_data in data_arr:
            names = ['%s_%s_%s' % (obj_data["i"], groupby_col, col) for col in cols]            
            all_data[names] = (obj_data["d"]).astype("float32")
        all_data.reset_index(inplace=True)
    return all_data

def group_by_date_countd(all_data):
    groupby_col = "date"
    unique_products = all_data.groupby([groupby_col])["eqt_code"].nunique()
    all_data.set_index([groupby_col], inplace=True)
    all_data["countd_product"] = unique_products.astype('uint16')
    all_data.reset_index(inplace=True)
    return all_data


def group_by_product_countd(all_data):
    groupby_col = "eqt_code"
    unique_date = all_data.groupby([groupby_col])["date"].nunique()

    all_data.set_index([groupby_col], inplace=True)
    all_data["countd_date"] = unique_date.astype('uint16')
    all_data.reset_index(inplace=True)
    return all_data

In [21]:
#print(all_data.head())

In [ ]:
#group by date and product_id to get more features
calculation_group_by =[
    {"id":"date" ,
     "cols": return_cols,
    },
    {"id":"eqt_code" ,
     "cols": return_cols,
    }
]
train_df["is_train"] = True
test_df["is_train"] = False
test_df["TARGET"] = None
all_data = pd.concat([train_df, test_df])
del train_df
del test_df

print("Creating new aggregated features")
all_data = get_mean_groupby(all_data, calculation_group_by)
print("Group by date")
all_data = group_by_date_countd(all_data)
print("Group by product")
all_data = group_by_product_countd(all_data)

Creating new aggregated features
date
eqt_code


In [ ]:
#replace nan per 0
rep_values= 0
all_data = all_data.replace([np.inf, -np.inf], rep_values)
all_data.fillna(rep_values, inplace=True)

In [ ]:
#file_name = "all_data.pkl"
#all_data.to_pickle(file_name)

## Prepare to Train

In [33]:
all_data = pd.read_pickle("all_data.pkl")

In [34]:
return_cols = all_data.columns.values[2:73]
other_cols = ["date" , "eqt_code"]
print(return_cols)

['09:30:00' '09:35:00' '09:40:00' '09:45:00' '09:50:00' '09:55:00'
 '10:00:00' '10:05:00' '10:10:00' '10:15:00' '10:20:00' '10:25:00'
 '10:30:00' '10:35:00' '10:40:00' '10:45:00' '10:50:00' '10:55:00'
 '11:00:00' '11:05:00' '11:10:00' '11:15:00' '11:20:00' '11:25:00'
 '11:30:00' '11:35:00' '11:40:00' '11:45:00' '11:50:00' '11:55:00'
 '12:00:00' '12:05:00' '12:10:00' '12:15:00' '12:20:00' '12:25:00'
 '12:30:00' '12:35:00' '12:40:00' '12:45:00' '12:50:00' '12:55:00'
 '13:00:00' '13:05:00' '13:10:00' '13:15:00' '13:20:00' '13:25:00'
 '13:30:00' '13:35:00' '13:40:00' '13:45:00' '13:50:00' '13:55:00'
 '14:00:00' '14:05:00' '14:10:00' '14:15:00' '14:20:00' '14:25:00'
 '14:30:00' '14:35:00' '14:40:00' '14:45:00' '14:50:00' '14:55:00'
 '15:00:00' '15:05:00' '15:10:00' '15:15:00' '15:20:00']


In [35]:
#get all cols per type
#TODO refactor to do it in a for loop 

return_avg_date_cols = ["avg_date_%s" % x for x in return_cols]
return_avg_product_cols = ["avg_eqt_code_%s" % x for x in return_cols]

return_std_date_cols = ["std_date_%s" % x for x in return_cols]
return_std_product_cols = ["std_eqt_code_%s" % x for x in return_cols]

return_median_date_cols = ["median_date_%s" % x for x in return_cols]
return_median_product_cols = ["median_eqt_code_%s" % x for x in return_cols]

return_kurt_date_cols = ["kurt_date_%s" % x for x in return_cols]
return_kurt_product_cols = ["kurt_eqt_code_%s" % x for x in return_cols]

return_nan_date_cols = ["nan_date_%s" % x for x in return_cols]
return_nan_product_cols = ["nan_eqt_code_%s" % x for x in return_cols]

In [45]:
#encode and preprocess data

def fit_encoder(obj_in_cols_obj, all_data, encoder_name="StandardScaler"):
    if encoder_name is not None:
        for i,x in enumerate(obj_in_cols_obj):
            group_cols = x["cols"] #group of lstm cols
            encoders = []
            for cols in group_cols:
                X_all = all_data[cols].values
                encoder = df_mapper(cols, X_all, encoder_name)
                encoders.append(encoder)
            obj_in_cols_obj[i]["encoders"] = encoders
    return obj_in_cols_obj

def df_mapper(cols, all_data , encoder):
    if encoder == "StandardScaler":
        encoder = StandardScaler() 
    elif encoder == "LabelEncoder":
        encoder = LabelEncoder()
    elif encoder == "RobustScaler":
        encoder = RobustScaler()
    elif encoder == "MinMaxScaler":
        encoder = MinMaxScaler(feature_range=(0,1))
    else:
        raise NotFoundException()
        
    encoder = encoder.fit(all_data)
    return encoder
 
def prepare_data(lstm_in_cols_obj, emb_in_cols_obj, df_to_prepare):
    def __prepare_data_inn(cols, df_tmp,  encoders, encode=True, reshape=True):
        Xs = []
        for i, col in enumerate(cols):
            X = df_tmp[col].values # timeseries cols
            if encode and encoders[i] is not None:
                X = encoders[i].transform(X)
            if reshape:
                X = X.reshape(X.shape[0], X.shape[1], 1)
            Xs.append(X)
        X = np.concatenate([x for x in Xs] , axis=len(Xs[0].shape)-1) if len(Xs) > 1 else Xs[0] 
        return X
       
    Xs_map = []
    for x in lstm_in_cols_obj:
        cols = x["cols"]
        encoders = x["encoders"] if "encoders" in x else None
        encode = True if "encoders" in x else False
        Xs_map.append(__prepare_data_inn(cols, df_tmp=df_to_prepare, reshape=True, encoders=encoders, encode=encode))
    #TODO Refactor ravel
    embs = []
    for x in emb_in_cols_obj:
        cols = x["cols"]
        encoders = x["encoders"] if "encoders" in x else None
        encode = True if "encoders" in x else False
        X_emb = __prepare_data_inn(cols, df_tmp=df_to_prepare, reshape=False, encoders=encoders, encode=encode)
        embs.append(X_emb)
    X_t = [x for x in Xs_map] + embs

    return X_t


def generate(df_generator, target, batch_size, lstm_in_cols_obj,emb_in_cols_obj, y_vals=True, shuffle=True):
    imax = int(math.ceil((df_generator.shape[0])/batch_size))
    while True:
        indexes = __get_exploration_order(df_generator, shuffle)
        for i in range(imax):
            upp = min((i+1)*batch_size, df_generator.shape[0])
            df_tmp = df_generator.loc[indexes[i*batch_size:upp]]
            X = prepare_data(lstm_in_cols_obj,emb_in_cols_obj, df_tmp)
            if y_vals:
                y = (df_tmp[target].values >= 0)
                yield X, y
            else:
                yield X

                
def __get_exploration_order(df, shuffle=True):
    indexes = df.index.values
    if shuffle == True:
        np.random.shuffle(indexes)
    return indexes


In [37]:
#create the model
def create_model(lstm_in_cols_obj, emb_in_cols_obj):
    def create_emb(name, out_len, in_len):
        inp = Input(shape=[1], dtype='int64', name="%s_in" % name)
        out = (Embedding(in_len, out_len, name="%s_emb" % name)(inp))
        out = SpatialDropout1D(0.2)(out)
        out = Flatten()(out)
        return inp,out


    def create_lstm(name, lstm_out, timesteps_in, dims_in):
        model_inp = Input(shape=(timesteps_in,dims_in), name="%s_in" % name)
        model_out = LSTM(lstm_out,return_sequences=False,name="%s_lstm" % name)(model_inp)
        return model_inp, model_out

    lstm_in_array= []
    lstm_out_array = []
    for x in lstm_in_cols_obj:
        name = x["name"]
        dims_in = len(x["cols"])
        lstm_out = x["out_l"]
        x_model_in, x_model_out = create_lstm(name=name, lstm_out=lstm_out, timesteps_in=71, dims_in=dims_in)
        lstm_in_array.append(x_model_in)
        lstm_out_array.append(x_model_out)
    
    #add first two
    #model_lstm_add_two = add([lstm_out_array[0] , lstm_out_array[1]])
    #model_lstm_add_two = PReLU()(model_lstm_add_two)
    
    
    #add last three lstm
    #model_lstm_add_three = add([lstm_out_array[2] , lstm_out_array[3] , lstm_out_array[4]])
    #model_lstm_add_three = PReLU()(model_lstm_add_three)

    #add last two lstm
    model_lstm_add_two = add([lstm_out_array[1] , lstm_out_array[2]])
    model_lstm_add_two = PReLU()(model_lstm_add_two)
    
    #concat with add layers + origin layer
    model_lstm = concatenate([model_lstm_add_two] + lstm_out_array)
    model_lstm = (Dense(200)(model_lstm))
    model_lstm = PReLU()(model_lstm)

    #create embeddings
    in_array_emb= []
    out_array_emb = []
    for x in emb_in_cols_obj:  
        in_len = x["in_l"]
        out_len = x["out_l"]
        name = x["name"]
        inp_emb, out_emb = create_emb(name=name , out_len=out_len, in_len = in_len)
        in_array_emb.append(inp_emb)
        out_array_emb.append((out_emb))
        
    model_emb = concatenate(out_array_emb) if len(out_array_emb) > 1 else out_array_emb[0]
    model_emb = Dropout(0.2)(Dense(100)(model_emb))
    model_emb = PReLU()(model_emb)
    
    #concatenate embeddings & lstms
    model = concatenate([model_emb] + [model_lstm]+ [model_lstm_add_two])

    model = Dense(300)(model)
    model = PReLU()(model)
    model = Dropout(0.2)(model)
    model = Dense(1, activation="sigmoid")(model)
    model =  Model(lstm_in_array+in_array_emb, model)
    
    return model
    

Params for model

In [38]:
#params of the model
lstm_in_cols_obj = [
#{"name" : "volatility" , "cols": [volatility_cols] , "out_l": 45} ,
{"name" : "return" , "cols": [return_cols] , "out_l": 30} , #checker pour le 45 devient 30
{"name" : "return_per_product" , "cols": [return_avg_product_cols, return_median_product_cols, return_std_product_cols, return_nan_product_cols], "out_l": 15} ,
{"name" : "return_per_date" , "cols": [return_avg_date_cols, return_median_date_cols, return_std_date_cols, return_nan_date_cols], "out_l": 15} , 
#{"name" : "return_per_date_and_product" , "cols": [return_avg_date_cols, return_kurt_date_cols , return_avg_product_cols, return_kurt_product_cols ], "out_l": 15}
]

emb_in_cols_obj = [
    {"name" : "emb_eqt_code" , "cols": [["eqt_code"]] , "out_l": 80, "in_l" : all_data["eqt_code"].nunique()} ,
    {"name" : "countd_date" , "cols": [["countd_date"]] , "out_l": 10 , "in_l" : all_data["countd_date"].nunique()} ,
    {"name" : "countd_product" , "cols": [["countd_product"]] , "out_l": 5 , "in_l" : all_data["countd_product"].nunique()} ,
    {"name" : "countd_ret_nan" , "cols": [["ret_nan"]] , "out_l": 5 , "in_l" : all_data["ret_nan"].nunique()}
]

In [39]:
### add encoders in lstm_in_cols_obj 
lstm_in_cols_obj = fit_encoder(lstm_in_cols_obj, all_data , encoder_name="StandardScaler")
emb_in_cols_obj = fit_encoder(emb_in_cols_obj, all_data , encoder_name="LabelEncoder")

In [40]:
test_df = all_data[all_data["is_train"] == False]
train_df = all_data[all_data["is_train"] == True]
#del all_data;

In [ ]:
#train_df_samp = train_df.sample(5000)

In [41]:
X_test = prepare_data(lstm_in_cols_obj, emb_in_cols_obj, test_df)

In [ ]:
#X_test = prepare_data(lstm_in_cols_obj, emb_in_cols_obj, test_df)

In [42]:
model = create_model(lstm_in_cols_obj, emb_in_cols_obj)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
emb_eqt_code_in (InputLayer)    (None, 1)            0                                            
__________________________________________________________________________________________________
countd_date_in (InputLayer)     (None, 1)            0                                            
__________________________________________________________________________________________________
countd_product_in (InputLayer)  (None, 1)            0                                            
__________________________________________________________________________________________________
countd_ret_nan_in (InputLayer)  (None, 1)            0                                            
__________________________________________________________________________________________________
emb_eqt_co

### Train the model with 5 KFold 

In [43]:
#sort values in another order
#sort values per date in order to NOT train with dates (validation is done with dates never seen like in the test set) 
seed = 42
#train_df["shuffle"] = train_df['date'].apply(lambda x: hashlib.sha256(str(x*seed)).hexdigest())
train_df.sort_values(["date"], inplace=True)
monitor = "val_loss"
model_prefix = "cfm-example-model-%s" % datetime.datetime.now().strftime("%Y-%m-%d")
total_folds = 5
total_it = 120

In [47]:
kfold = KFold(n_splits=total_folds, shuffle=False)
cvscores = []
test_preds = []
batch_size = (len(train_df) - len(train_df)//total_folds)//(total_it)
i = 1
for train, valid in kfold.split(train_df):
    tmp_df_train = train_df.loc[train].reset_index(drop=True)
    tmp_df_valid = train_df.loc[valid].sample(frac=1).reset_index(drop=True)
    print("Train %d , Valid %d" % (len(tmp_df_train) , len(tmp_df_valid)))

    base_lr = 0.001
    j = 1 
    last_loss_score = 100 #default init.
    model = create_model(lstm_in_cols_obj, emb_in_cols_obj)
    once = False
    print("Start Fold %i - start base_lr %0.4f : " % (i , base_lr))
    while True:
        model_checkpoint_name_tmp = "weights/%s-best-fold-%d-it-%d.h5" % (model_prefix, i, j )
        callbacks = [EarlyStopping(monitor=monitor, patience=3, verbose=1, min_delta=1e-8, mode='min'),
                     ModelCheckpoint(model_checkpoint_name_tmp, monitor=monitor, save_best_only=True, save_weights_only=True, verbose=1, mode="min")
                    ]
        step_size = 100
        clr  = CyclicLR(mode='exp_range', max_lr=base_lr*6, base_lr=base_lr, step_size=step_size)
        callbacks.append(clr)
        
        train_gen = generate(tmp_df_train, target_col, batch_size, lstm_in_cols_obj, emb_in_cols_obj, True, True)
        valid_gen = generate(tmp_df_valid, target_col, batch_size, lstm_in_cols_obj, emb_in_cols_obj, True, False)
        
        model.compile(loss= "binary_crossentropy",                      
                  optimizer='Adam',
                      metrics=["accuracy"])
        hist = model.fit_generator(
            generator=train_gen,
            validation_data=valid_gen,
            steps_per_epoch = math.ceil((tmp_df_train.shape[0])/batch_size),
            validation_steps = math.ceil((tmp_df_valid.shape[0])/batch_size),
            verbose=1,epochs=50,
            callbacks = callbacks,
            max_queue_size=30, workers=1, #from 8 to 1
            use_multiprocessing=True,
            workers=multiprocessing.cpu_count()-1
                 )         
        model.load_weights(model_checkpoint_name_tmp)
        loss_score,score = model.evaluate_generator(valid_gen, steps=math.ceil((tmp_df_valid.shape[0])/batch_size))
        print("Checkpoint. Fold %i | Iteration %i (LR %0.6f) -- Score --> %0.8f | %0.8f "  % (i, j , base_lr , score,  loss_score))
        if loss_score > last_loss_score:
            print("Score Worse than before (%0.8f | %0.8f)" % ( loss_score , last_loss_score))
            if once:
                break
            once = True
            print("Try one last time reducing LR.")
            base_lr = base_lr/10

        else:
            once = False
            print("Score Better than before (%0.8f | %0.8f)" % (loss_score ,last_loss_score))
            last_loss_score=loss_score
            base_lr = base_lr/5
            best_j = j 
            print("Reducing LR to %f" % base_lr)
        if j >3:
            print("Stop. Too Much Iterations.")
            break
        model_checkpoint_name_tmp = "weights/%s-best-fold-%d-it-%d.h5" % (model_prefix, i, best_j )
        model.load_weights(model_checkpoint_name_tmp)
        j = j+1
            

    print("End of Training for Fold %i. Predicting."  % (i))
    #GET LAST SCORE
    model_checkpoint_name = "weights/%s-best-fold-%d-it-%d.h5" % (model_prefix, i, best_j )
    model.load_weights(model_checkpoint_name)
    #save best model
    model.save_weights("weights/%s-best-fold-%d-Best.h5" % (model_prefix, i) , overwrite=True )
    score,score_mae = model.evaluate_generator(valid_gen, steps=math.ceil(len(tmp_df_valid)/batch_size))

    test_yPreds = model.predict(X_test, batch_size=2048, verbose=0)
    cvscores.append((score_mae,score))
    test_preds.append(test_yPreds[:,-1])
    
    print("End of Fold %i | Score --> %0.4f | %0.4f"  % (i, score, score_mae) )
    i=i+1

Train 596261 , Valid 149066
Start Fold 1 - start base_lr 0.0010 : 
Epoch 1/50
121/121 [==============================] - 856s 7s/step - loss: 0.6929 - acc: 0.5123 - val_loss: 0.6929 - val_acc: 0.5096

Epoch 00001: val_loss improved from inf to 0.69292, saving model to weights/cfm-example-model-2019-02-28-best-fold-1-it-1.h5


OSError: Unable to create file (unable to open file: name = 'weights/cfm-example-model-2019-02-28-best-fold-1-it-1.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 602)

In [ ]:
cvscores

In [ ]:
mean_cv_score = np.mean(cvscores, axis=0)
mean_cv_score

In [ ]:
avg_test_score = np.mean(test_preds, axis=0)
out_name = 'results/%s-%d-Kfolds_%0.6f_%0.6f.csv' % (model_prefix, total_folds, mean_cv_score[0], mean_cv_score[1])
submission = pd.DataFrame({'ID':test_df["ID"], 'TARGET':avg_test_score.ravel()})
submission.to_csv(out_name , index=False)